In [4]:
pip install --upgrade --quiet  langchain langchain-experimental langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

SELECT COUNT(*) FROM Employee


In [5]:
db.run(response)

'[(8,)]'

In [11]:
def query_out(query):
    response = chain.invoke({"question":query})
    return print(response),db.run(response)    

In [12]:
query = "List the total sales per country for all the countries in database"
query_out(query)

SELECT "BillingCountry", SUM("Total") AS "TotalSales"
FROM "Invoice"
GROUP BY "BillingCountry"
ORDER BY "TotalSales" DESC


(None,
 "[('USA', 523.0600000000003), ('Canada', 303.9599999999999), ('France', 195.09999999999994), ('Brazil', 190.09999999999997), ('Germany', 156.48), ('United Kingdom', 112.85999999999999), ('Czech Republic', 90.24000000000001), ('Portugal', 77.23999999999998), ('India', 75.25999999999999), ('Chile', 46.62), ('Ireland', 45.62), ('Hungary', 45.62), ('Austria', 42.62), ('Finland', 41.620000000000005), ('Netherlands', 40.62), ('Norway', 39.62), ('Sweden', 38.620000000000005), ('Poland', 37.620000000000005), ('Italy', 37.620000000000005), ('Denmark', 37.620000000000005), ('Australia', 37.620000000000005), ('Argentina', 37.620000000000005), ('Spain', 37.62), ('Belgium', 37.62)]")

In [9]:
db_chain.run("Which state has maximum no. of customer above age 50 years?")



> Entering new SQLDatabaseChain chain...
Which state has maximum no. of customer above age 50 years?
SQLQuery:SELECT Customer.State, COUNT(Customer.CustomerId) AS "No. of Customers"
FROM Customer
INNER JOIN Employee ON Customer.SupportRepId = Employee.EmployeeId
WHERE Employee.BirthDate <= date('now', '-50 years')
GROUP BY Customer.State
ORDER BY COUNT(Customer.CustomerId) DESC
LIMIT 1
SQLResult: [(None, 29)]
Answer:None
> Finished chain.


'None'

In [10]:
db_chain.run("Which state has maximum no. of customer below age 50 years?")



> Entering new SQLDatabaseChain chain...
Which state has maximum no. of customer below age 50 years?
SQLQuery:SELECT Customer.State, COUNT(Customer.CustomerId) AS "No. of Customers"
FROM Customer
INNER JOIN Employee ON Customer.SupportRepId = Employee.EmployeeId
WHERE Employee.BirthDate > date('now', '-50 years')
GROUP BY Customer.State
ORDER BY COUNT(Customer.CustomerId) DESC
LIMIT 1
SQLResult: 
Answer:The state with the maximum number of customers below the age of 50 is Alberta.
> Finished chain.


'The state with the maximum number of customers below the age of 50 is Alberta.'

In [11]:
db_chain.run("Top 5 customer name with highest Invoice Total")



> Entering new SQLDatabaseChain chain...
Top 5 customer name with highest Invoice Total
SQLQuery:SELECT c.FirstName || " " || c.LastName AS "Customer Name", SUM(i.Total) AS "Total Invoice"
FROM Customer c
INNER JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY "Total Invoice" DESC
LIMIT 5
SQLResult: [('Helena Holý', 49.620000000000005), ('Richard Cunningham', 47.620000000000005), ('Luis Rojas', 46.62), ('Ladislav Kovács', 45.62), ("Hugh O'Reilly", 45.62)]
Answer:Helena Holý
> Finished chain.


'Helena Holý'

In [12]:
db_chain.run("List the total sales per country. Which country's customers spent the most?")



> Entering new SQLDatabaseChain chain...
List the total sales per country. Which country's customers spent the most?
SQLQuery:SELECT c.Country, SUM(i.Total) AS "Total Sales"
FROM Customer c
INNER JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.Country
ORDER BY "Total Sales" DESC
LIMIT 1;
SQLResult: [('USA', 523.0600000000003)]
Answer:USA
> Finished chain.


'USA'

In [15]:
db_chain.run("List the total sales per country for all the countries in database without limiting it to specific number")



> Entering new SQLDatabaseChain chain...
List the total sales per country for all the countries in database without limiting it to specific number
SQLQuery:SELECT c.Country, SUM(i.Total) AS "Total Sales"
FROM Customer c
INNER JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.Country
ORDER BY "Total Sales" DESC
SQLResult: [('USA', 523.0600000000003), ('Canada', 303.9599999999999), ('France', 195.09999999999994), ('Brazil', 190.09999999999997), ('Germany', 156.48), ('United Kingdom', 112.85999999999999), ('Czech Republic', 90.24000000000001), ('Portugal', 77.23999999999998), ('India', 75.25999999999999), ('Chile', 46.62), ('Ireland', 45.62), ('Hungary', 45.62), ('Austria', 42.62), ('Finland', 41.620000000000005), ('Netherlands', 40.62), ('Norway', 39.62), ('Sweden', 38.620000000000005), ('Poland', 37.620000000000005), ('Italy', 37.620000000000005), ('Denmark', 37.620000000000005), ('Australia', 37.620000000000005), ('Argentina', 37.620000000000005), ('Spain', 37.62), ('Belgium',

'The total sales per country for all the countries in the database are: \n1. USA - $523.06\n2. Canada - $303.96\n3. France - $195.10\n4. Brazil - $190.10\n5. Germany - $156.48\n6. United Kingdom - $112.86\n7. Czech Republic - $90.24\n8. Portugal - $77.24\n9. India - $75.26\n10. Chile - $46.62\n11. Ireland - $45.62\n12. Hungary - $45.62\n13. Austria - $42.62\n14. Finland - $41.62\n15. Netherlands - $40.62\n16. Norway - $39.62\n17. Sweden - $38.62\n18. Poland - $37.62\n19. Italy - $37.62\n20. Denmark - $37.62\n21. Australia - $37.62\n22. Argentina - $37.62\n23. Spain - $37.62\n24. Belgium - $37.62'

In [16]:
db_chain.run("List all the tables in database")



> Entering new SQLDatabaseChain chain...
List all the tables in database
SQLQuery:SELECT name FROM sqlite_master WHERE type='table'
SQLResult: [('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
Answer:Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
> Finished chain.


'Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track'

In [17]:
db_chain.run("Describe Invoice Table")



> Entering new SQLDatabaseChain chain...
Describe Invoice Table
SQLQuery:SELECT "InvoiceId", "CustomerId", "InvoiceDate", "BillingAddress", "BillingCity", "BillingState", "BillingCountry", "BillingPostalCode", "Total" FROM "Invoice" LIMIT 5
SQLResult: [(1, 2, '2009-01-01 00:00:00', 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', 1.98), (2, 4, '2009-01-02 00:00:00', 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', 3.96), (3, 8, '2009-01-03 00:00:00', 'Grétrystraat 63', 'Brussels', None, 'Belgium', '1000', 5.94), (4, 14, '2009-01-06 00:00:00', '8210 111 ST NW', 'Edmonton', 'AB', 'Canada', 'T6G 2C7', 8.91), (5, 23, '2009-01-11 00:00:00', '69 Salem Street', 'Boston', 'MA', 'USA', '2113', 13.86)]
Answer:The Invoice table contains information about customer invoices, including the invoice ID, customer ID, invoice date, billing address, billing city, billing state, billing country, billing postal code, and total amount.
> Finished chain.


'The Invoice table contains information about customer invoices, including the invoice ID, customer ID, invoice date, billing address, billing city, billing state, billing country, billing postal code, and total amount.'